Setup and imports

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras.layers import Dense, Flatten , Dropout, Conv2D, MaxPooling2D, BatchNormalization, MaxPool2D, GlobalAvgPool2D
from keras.models import Sequential
from keras.regularizers import l2
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.applications import ResNet50, ResNet50V2, InceptionV3, Xception, ResNet152, ResNet152V2
from keras.preprocessing.image import ImageDataGenerator
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import numpy as np
import random
import cv2
import os
import concurrent.futures
import matplotlib.pyplot as plt

Get input data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
path = '/content/drive/MyDrive/Dataset/raw-img'
image_size = 32
class_names = sorted(os.listdir(path))

def process_animal(animal):
    animal_path = path + "/" + animal
    animal_label = animal
    formatted_data = []
    print(f"Started processing animal: {animal_label}")  # Debug statement
    for animal_image in os.listdir(animal_path):
        formatted_array = []

        img = cv2.imread(animal_path + "/" + animal_image)
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        formatted_array = cv2.resize(img_rgb, (image_size, image_size))
        formatted_data.append([formatted_array, animal_label])
    print(f"Finished processing animal: {animal_label}")  # Debug statement
    return formatted_data

with concurrent.futures.ThreadPoolExecutor() as executor:
    results = list(executor.map(process_animal, class_names))
    print("All tasks have completed.")  # Debug statement

# Flatten the list of results
formatted_data = [item for sublist in results for item in sublist]

Started processing animal: butterflyStarted processing animal: cat

Started processing animal: chicken
Started processing animal: cow
Started processing animal: dog
Started processing animal: elephant
Finished processing animal: elephant
Started processing animal: horse
Finished processing animal: horse
Started processing animal: sheep
Finished processing animal: sheep
Started processing animal: spider
Finished processing animal: spider
Started processing animal: squirrel
Finished processing animal: squirrel
Finished processing animal: cat
Finished processing animal: cow
Finished processing animal: butterfly
Finished processing animal: chicken
Finished processing animal: dog
All tasks have completed.


In [ ]:
random.shuffle(formatted_data)
x_data = []
y_labels = []

data_gen = ImageDataGenerator(
      rotation_range=20,
      width_shift_range=0.1,
      height_shift_range=0.1,
      rescale=1./255,
      shear_range=0.1,
      zoom_range=0.1,
      horizontal_flip=True,
      fill_mode='nearest')

for image, label in formatted_data:
  x_data.append(image)
  y_labels.append(label)

encoder = LabelEncoder()

train_data, test_data, train_labels, test_labels = train_test_split(x_data, y_labels, test_size=0.2)

train_data, val_data, train_labels, val_labels = train_test_split(train_data, train_labels, test_size=0.2)

train_labels = encoder.fit_transform(train_labels)
test_labels = encoder.transform(test_labels)
val_labels = encoder.transform(val_labels)

x_train = np.array(train_data).reshape(-1,image_size,image_size,3)
x_train = tf.keras.utils.normalize(x_train,axis=1)

y_train = np.array(train_labels)
x_train = data_gen.flow(x_train, y_train, shuffle=True)

val_data = np.array(val_data).reshape(-1,image_size,image_size,3)
val_data = tf.keras.utils.normalize(val_data,axis=1)

test_data = np.array(test_data).reshape(-1,image_size,image_size,3)
test_data = tf.keras.utils.normalize(test_data,axis=1)

NameError: name 'random' is not defined

Building the model

In [ ]:
model = Sequential()
model.add(Conv2D(64, 3, activation='relu', input_shape=(image_size, image_size, 3), kernel_regularizer=l2(0.08), padding='same'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(MaxPool2D(pool_size=2, strides=2))

# model.add(Flatten())
model.add(GlobalAveragePooling2D())

# model.add(Dense(128, activation='relu', kernel_regularizer=l2(0.03)))
# model.add(Dropout(0.4))
# model.add(BatchNormalization())

model.add(Dense(64, activation='relu', kernel_regularizer=l2(0.03)))
model.add(Dropout(0.4))
model.add(BatchNormalization())

model.add(Dense(10, activation='softmax'))

In [ ]:
# Specify Model Name
name = "ResNet152V2"

# Pretrained Model
base_model = ResNet152V2(include_top=False, input_shape=(64,64,3), weights='imagenet')
base_model.trainable = True # Freeze the Weights

# Model
resnet152V2 = Sequential([
    base_model,
    GlobalAvgPool2D(),
    Dense(256, activation='relu', kernel_regularizer=l2(0.01)),
    Dropout(0.5),
    Dense(64, activation='relu', kernel_regularizer=l2(0.01)),
    Dropout(0.5),
    Dense(10, activation='softmax')
], name=name)

# Compile
resnet152V2.compile(
    loss='sparse_categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

# Callbacks
cbs = [
    EarlyStopping(patience=3, restore_best_weights=True, verbose=1),
    ModelCheckpoint(name + ".h5", save_best_only=True, verbose=1)
]

# Train Model
history = resnet152V2.fit(
    x_train, validation_data=(val_data, val_labels),
    epochs=20, callbacks=cbs
)

Epoch 1/20
 73/524 [===>..........................] - ETA: 1:18:22 - loss: 6.2810 - accuracy: 0.1734

KeyboardInterrupt: 

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 32, 32, 64)        1792      
                                                                 
 batch_normalization (Batch  (None, 32, 32, 64)        256       
 Normalization)                                                  
                                                                 
 dropout (Dropout)           (None, 32, 32, 64)        0         
                                                                 
 max_pooling2d (MaxPooling2  (None, 16, 16, 64)        0         
 D)                                                              
                                                                 
 flatten (Flatten)           (None, 16384)             0         
                                                                 
 dense (Dense)               (None, 64)                1

Compiling the model

In [ ]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics='accuracy')

In [ ]:
call = [
    EarlyStopping(patience=6, restore_best_weights=True, verbose=1),
    ModelCheckpoint("best.h5", save_best_only=True, verbose=1)
]

In [ ]:
history = model.fit(x_train, epochs=30, validation_data=(val_data, val_labels), callbacks=call, batch_size=50)

Epoch 1/30
524/524 [==============================] - ETA: 0s - loss: 2.4031 - accuracy: 0.2160
Epoch 1: val_loss improved from inf to 32.16228, saving model to best.h5
524/524 [==============================] - 76s 141ms/step - loss: 2.4031 - accuracy: 0.2160 - val_loss: 32.1623 - val_accuracy: 0.1083
Epoch 2/30


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


524/524 [==============================] - ETA: 0s - loss: 2.2408 - accuracy: 0.2388
Epoch 2: val_loss did not improve from 32.16228
524/524 [==============================] - 69s 133ms/step - loss: 2.2408 - accuracy: 0.2388 - val_loss: 207.5480 - val_accuracy: 0.1930
Epoch 3/30
524/524 [==============================] - ETA: 0s - loss: 2.2276 - accuracy: 0.2419
Epoch 3: val_loss did not improve from 32.16228
524/524 [==============================] - 67s 128ms/step - loss: 2.2276 - accuracy: 0.2419 - val_loss: 101.4166 - val_accuracy: 0.2073
Epoch 4/30
524/524 [==============================] - ETA: 0s - loss: 2.2222 - accuracy: 0.2404
Epoch 4: val_loss did not improve from 32.16228
524/524 [==============================] - 65s 123ms/step - loss: 2.2222 - accuracy: 0.2404 - val_loss: 89.8657 - val_accuracy: 0.2019
Epoch 5/30
524/524 [==============================] - ETA: 0s - loss: 2.2058 - accuracy: 0.2441
Epoch 5: val_loss did not improve from 32.16228
524/524 [===================

In [ ]:
loss, accuracy = model.evaluate(test_data, test_labels)

print(f'Test accuracy: {accuracy}')

In [ ]:
plt.figure()
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()